## 1 Load Dataset

In [11]:
import pandas as pd

customers = pd.read_csv(r"C:\Users\msiza\Downloads\Customers.csv")
products = pd.read_csv(r"C:\Users\msiza\Downloads\Products.csv")
transactions = pd.read_csv(r"C:\Users\msiza\Downloads\Transactions.csv")


## 2. Merge datasets

In [12]:
df = transactions.merge(customers, on='CustomerID').merge(products, on='ProductID')


In [13]:
df

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price_x,CustomerName,Region,SignupDate,ProductName,Category,Price_y
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68,Andrea Jenkins,Europe,2022-12-03,ComfortLiving Bluetooth Speaker,Electronics,300.68
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68,Brittany Harvey,Asia,2024-09-04,ComfortLiving Bluetooth Speaker,Electronics,300.68
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68,Kathryn Stevens,Europe,2024-04-04,ComfortLiving Bluetooth Speaker,Electronics,300.68
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68,Travis Campbell,South America,2024-04-11,ComfortLiving Bluetooth Speaker,Electronics,300.68
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68,Timothy Perez,Europe,2022-03-15,ComfortLiving Bluetooth Speaker,Electronics,300.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,T00496,C0118,P037,2024-10-24 08:30:27,1,459.86,459.86,Jacob Holt,South America,2022-01-22,SoundWave Smartwatch,Electronics,459.86
996,T00759,C0059,P037,2024-06-04 02:15:24,3,1379.58,459.86,Mrs. Kimberly Wright,North America,2024-04-07,SoundWave Smartwatch,Electronics,459.86
997,T00922,C0018,P037,2024-04-05 13:05:32,4,1839.44,459.86,Tyler Haynes,North America,2024-09-21,SoundWave Smartwatch,Electronics,459.86
998,T00959,C0115,P037,2024-09-29 10:16:02,2,919.72,459.86,Joshua Hamilton,Asia,2024-11-11,SoundWave Smartwatch,Electronics,459.86


## 3.Data Preprocessing & Feature Engineering

In [14]:
df['TransactionDate'] = pd.to_datetime(df['TransactionDate'])
df['SignupDate'] = pd.to_datetime(df['SignupDate'])
df['CustomerTenure'] = (df['TransactionDate'].max() - df['SignupDate']).dt.days

customer_features = df.groupby('CustomerID').agg({
    'TotalValue': 'sum',
    'TransactionID': 'count',
    'Category': lambda x: x.mode()[0],  # Most frequently bought category
    'CustomerTenure': 'first'
}).reset_index()


## 4. One-Hot Encoding of Categories

In [15]:
customer_features = pd.get_dummies(customer_features, columns=['Category'])


In [16]:
customer_features

,CustomerID,TotalValue,TransactionID,CustomerTenure,Category_Books,Category_Clothing,Category_Electronics,Category_Home Decor
0,C0001,3354.52,5,902,False,False,True,False
1,C0002,1862.74,4,1049,False,True,False,False
2,C0003,2725.38,4,296,False,False,False,True
3,C0004,5354.88,8,811,True,False,False,False
4,C0005,2034.24,3,866,False,False,True,False
...,...,...,...,...,...,...,...,...
194,C0196,4982.88,4,935,False,False,False,True
195,C0197,1928.65,3,648,False,False,True,False
196,C0198,931.83,2,1035,False,True,False,False
197,C0199,1979.28,4,756,False,False,True,False


## 5. Standardization of Features

In [17]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
customer_features_scaled = scaler.fit_transform(customer_features.drop(columns=['CustomerID']))


## 6. Cosine Similarity Calculation &  Define Function to Get Top 3 Similar Customers

In [18]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Compute similarity matrix
similarity_matrix = cosine_similarity(customer_features_scaled)

# Convert to DataFrame
similarity_df = pd.DataFrame(similarity_matrix, index=customer_features['CustomerID'], columns=customer_features['CustomerID'])

# Function to get top 3 lookalikes
def get_top_3_similar(customers):
    results = {}
    for customer_id in customers:
        similar_customers = similarity_df.loc[customer_id].nlargest(4).iloc[1:]  # Exclude self
        results[customer_id] = list(zip(similar_customers.index, np.round(similar_customers.values, 2)))[:3]
    return results

# Get lookalikes for the first 20 customers
target_customers = customer_features['CustomerID'][:20].tolist()
lookalikes = get_top_3_similar(target_customers)


## 7. Get Lookalikes for the First 20 Customers

In [19]:
lookalikes

{'C0001': [('C0192', 0.94), ('C0184', 0.94), ('C0091', 0.93)],
 'C0002': [('C0029', 1.0), ('C0176', 0.98), ('C0103', 0.98)],
 'C0003': [('C0052', 1.0), ('C0178', 0.99), ('C0031', 0.99)],
 'C0004': [('C0173', 1.0), ('C0165', 0.98), ('C0175', 0.97)],
 'C0005': [('C0112', 1.0), ('C0007', 0.99), ('C0192', 0.99)],
 'C0006': [('C0064', 0.96), ('C0185', 0.94), ('C0187', 0.93)],
 'C0007': [('C0112', 0.99), ('C0005', 0.99), ('C0192', 0.97)],
 'C0008': [('C0065', 0.88), ('C0124', 0.88), ('C0012', 0.87)],
 'C0009': [('C0077', 1.0), ('C0058', 0.96), ('C0061', 0.95)],
 'C0010': [('C0062', 0.99), ('C0061', 0.97), ('C0029', 0.96)],
 'C0011': [('C0174', 0.97), ('C0135', 0.95), ('C0125', 0.95)],
 'C0012': [('C0124', 0.98), ('C0136', 0.96), ('C0195', 0.96)],
 'C0013': [('C0143', 1.0), ('C0093', 0.95), ('C0022', 0.94)],
 'C0014': [('C0151', 0.99), ('C0110', 0.99), ('C0080', 0.99)],
 'C0015': [('C0123', 1.0), ('C0063', 0.99), ('C0032', 0.96)],
 'C0016': [('C0183', 1.0), ('C0056', 0.98), ('C0022', 0.96)],


In [20]:
# Save the lookalikes to Lookalike.csv
lookalike_data = []
for customer_id, similar_customers in lookalikes.items():
    lookalike_data.append([customer_id, similar_customers])

# Create DataFrame for CSV output
lookalike_df = pd.DataFrame(lookalike_data, columns=['CustomerID', 'Top_3_Lookalikes'])
lookalike_df.to_csv(r'C:\Users\msiza\Downloads\msizaa_Sahane_Lookalike.csv', index=False)

# Display the Lookalike DataFrame
lookalike_df

,CustomerID,Top_3_Lookalikes
0,C0001,"[(C0192, 0.94), (C0184, 0.94), (C0091, 0.93)]"
1,C0002,"[(C0029, 1.0), (C0176, 0.98), (C0103, 0.98)]"
2,C0003,"[(C0052, 1.0), (C0178, 0.99), (C0031, 0.99)]"
3,C0004,"[(C0173, 1.0), (C0165, 0.98), (C0175, 0.97)]"
4,C0005,"[(C0112, 1.0), (C0007, 0.99), (C0192, 0.99)]"
5,C0006,"[(C0064, 0.96), (C0185, 0.94), (C0187, 0.93)]"
6,C0007,"[(C0112, 0.99), (C0005, 0.99), (C0192, 0.97)]"
7,C0008,"[(C0065, 0.88), (C0124, 0.88), (C0012, 0.87)]"
8,C0009,"[(C0077, 1.0), (C0058, 0.96), (C0061, 0.95)]"
9,C0010,"[(C0062, 0.99), (C0061, 0.97), (C0029, 0.96)]"
